In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/Shared\ drives/TFM_Clara/6_robot-surgery-segmentation-master

/content/drive/Shared drives/TFM_Clara/6_robot-surgery-segmentation-master


In [ ]:
!pip3 install albumentations

     |████████████████████████████████| 634kB 9.5MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654021 sha256=89dfa2a9e5ad4f1c2bf39d58583bf049e4d07c31725267dd4a8a77716b6a23d0
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [ ]:
!pip install torchvision

In [ ]:
!sh train_ft.sh

linknet
Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100% 83.3M/83.3M [00:02<00:00, 34.2MB/s]
2632
num train = 1579, num_val = 1053
Restored model, epoch 21, step 1,700
Epoch 21, lr 0.0001:   0% 0/1580 [00:31<?, ?it/s]
Ctrl+C, saving snapshot
done.
^C


In [ ]:
from torch import nn
from loss import LossBinary
from torch.optim import Adam
from torch.utils.data import DataLoader
%matplotlib inline
from pylab import *
import cv2
import torch
from dataset_ft import load_image
from dataset_ft import load_mask
from utils import cuda
from generate_masks import get_model
from albumentations import Compose, Normalize
from albumentations.pytorch.functional import img_to_tensor
from validation import validation_binary
from dataset import RoboticsDataset
import glob
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset  # For custom data-sets
from torchvision import transforms
from torchsummary import summary
rcParams['figure.figsize'] = 10, 10

In [ ]:
def img_transform(p=1):
    return Compose([
        Normalize(p=1)
    ], p=p)

In [ ]:
def mask_overlay(image, mask, color=(0, 255, 0)):
    """
    Helper function to visualize mask on the top of the car
    """
    mask = np.dstack((mask, mask, mask)) * np.array(color)
    mask = mask.astype(np.uint8)
    weighted_sum = cv2.addWeighted(mask, 0.5, image, 0.5, 0.)
    img = image.copy()
    ind = mask[:, :, 1] > 0    
    img[ind] = weighted_sum[ind]    
    return img

In [ ]:
root = !pwd
root = root[0]
root

'/content/drive/Shareddrives/TFM_Clara/6_robot-surgery-segmentation-master'

In [ ]:
model_path = 'data/models/linknet_binary_20/model_0.pt'
model = get_model(model_path, model_type='LinkNet34', problem_type='binary')

In [ ]:
from train_ft import train

jaccard_weight = 0.3

root = !pwd
root = root[0]
batch_size = 2
n_epochs = 20
lr1 = 0.0001
lr2 = 0.00001
num_workers = 12
train_size = 0.6
train_crop_height = 1056
train_crop_width = 1440
val_crop_height = 1056
val_crop_width = 1440
fold = ["train_data/video9_raw","train_data/video6_raw","train_data/video39_raw"]
fold = ["train_data/video9_raw"]
heights = [1056,1056,1056]
widths = [1920,1920,1440]
for i in range(len(fold)):
  print(fold[i])
  train(model, "linknet", 0, "ent", jaccard_weight, fold[i], root, batch_size, n_epochs+50+20*i, lr1, num_workers, 1, train_size, heights[i], widths[i], heights[i], widths[i])
  train(model, "linknet", 0, "ent", jaccard_weight, fold[i], root, batch_size, n_epochs+60+20*i, lr2, num_workers, 1, train_size, heights[i], widths[i], heights[i], widths[i])
